Volgende stappen
- los probleem met JSON serializable op: SegmentedText is niet JSON serializable. DONE: subclass JSONEncoder.
- maak Flask based service, die weer asearch functies aanroept - DONE
- schrijf handler functies voor aservice REST calls, in de aservice package, op basis huidige functies

In [1]:
import sys
sys.path.append('../packages')

from textservice import segmentedtext

In [2]:
import lxml
from lxml import etree
import uuid

path = '../data/tei-samples/bosb002graa04_01.xml'
datadir = '../data/'

_last_page_begin_index = 0
_last_section_begin_index = -1
_last_chapter_begin_index = -1
_last_paragraph_begin_index = -1
_last_head_begin_index = -1

_last_page_end_index = -1
_last_section_end_index = -1
_last_chapter_end_index = -1
_last_paragraph_end_index = -1
_last_head_end_index = -1

_last_page_id = ""

all_textelements=segmentedtext.SplittableSegmentedText()
all_annotations=[]

def get_file_sequence_for_container(text_container):
    return [path]

def get_root_tree_element(file):
    # use iterparse to traverse the xml hierarchy, depth first, post order
    return etree.iterparse(path, events=('start','end'))

# handle each of the elements in the hierarchy according to 'layer type'
def handle_element(action,e,text,annotations):  
    global _last_page_begin_index
    global _last_section_begin_index
    global _last_chapter_begin_index
    global _last_paragraph_begin_index
    global _last_head_begin_index

    global _last_page_end_index
    global _last_section_end_index
    global _last_chapter_end_index
    global _last_paragraph_end_index
    global _last_head_end_index
    
    global _last_page_id

    if action == 'start':
        # store last begin indexes
        if e.tag == 'p':
            _last_paragraph_begin_index = text.len()               
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_begin_index = text.len()
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_begin_index = text.len()
        elif e.tag == 'head':
            _last_head_begin_index = text.len()
    elif action == 'end':
        if e.tag == 'p': 
            # leaf text element, add to all_textelements, also include text after possible pb's
            for index, t in enumerate(e.itertext()):
                text.append(t.strip())
                if index > 0: # assume: caused by pb contained within p. Update page end.
                    _last_page_end_index = text.len()-1
            
            _last_paragraph_end_index = text.len()-1

            if _last_paragraph_begin_index <= _last_paragraph_end_index:
                annotations.append({'label':'paragraph','begin_anchor': text._anchors[_last_paragraph_begin_index],\
                            'end_anchor':text._anchors[_last_paragraph_end_index],'id': 'annot_'+str(uuid.uuid4())})
        elif e.tag == 'head':
            # leaf text element, add to all_textelements
            text.append(e.text)
            
            _last_head_end_index = text.len()-1
            annotations.append({'label':'head','begin_anchor': text._anchors[_last_head_begin_index],\
                            'end_anchor':text._anchors[_last_head_end_index],'id': 'annot_'+str(uuid.uuid4())}) 
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_end_index = text.len()-1
            annotations.append({'label':'chapter','begin_anchor': text._anchors[_last_chapter_begin_index],\
                            'end_anchor':text._anchors[_last_chapter_end_index],'id': 'annot_'+str(uuid.uuid4())})            
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_end_index = text.len()-1
            annotations.append({'label':'section','begin_anchor': text._anchors[_last_section_begin_index],\
                            'end_anchor':text._anchors[_last_section_end_index],'id': 'annot_'+str(uuid.uuid4())})               
        elif e.tag == 'pb':
            # first store the 'previous' page, then store begin and end of currently closed page
            annotations.append({'label':'page','begin_anchor': text._anchors[_last_page_begin_index],\
                            'end_anchor':text._anchors[_last_page_end_index],'id': _last_page_id}) 
            _last_page_begin_index = _last_page_end_index
            _last_page_end_index = text.len()-1 
            _last_page_id = f"page-{e.get('n')}"
            
    return        

def traverse(node,node_label,text,annotations):
    for action, elem in node:
        handle_element(action,elem,text,annotations)  
    return
    
# Process per file, properly concatenate results, maintaining proper referencing the baseline text elements
for f_name in get_file_sequence_for_container('just one tei document'):
    text_segments = segmentedtext.SplittableSegmentedText()
    annotation_array = []
            
    source_data = get_root_tree_element(f_name)

    traverse(source_data,'',text_segments,annotation_array)
           
    # properly concatenate annotation info taking ongoing line indexes into account - trivial, do not apply in this case
#    for ai in annotation_array:
#        ai['begin_index'] += len(all_textlines)
#        ai['end_index'] += len(all_textlines)
    
    all_textelements.extend(text_segments)       
    all_annotations.extend(annotation_array)
    

In [3]:
all_textelements.len()

3130

In [4]:
# return generator for annotations with a specific label
#def get_annotations_of_type(type,annotations):
#    return (a for a in annotations if a['label'] == type)

from annotation import asearch

In [5]:
sample_para = [paras for paras in asearch.get_annotations_of_type('paragraph', all_annotations)][103]
print(sample_para['begin_anchor'])
print(sample_para['end_anchor'])

anchor_3f47cd5a-216c-48e5-b51f-8702b34e9bfb
anchor_3f47cd5a-216c-48e5-b51f-8702b34e9bfb


In [6]:
all_textelements.slice(sample_para['begin_anchor'], sample_para['end_anchor'])

['‘Die goede Geertrui!’ zeide Elisabeth glimlachend tot Courtenay; ‘zij is zoo ingenomen met den titel, dien mijn vader haar gaf, toen zij mijne min werd, dat zij zelfs hare gehechtheid aan mij onder de stijve windsels der étiquette verbergt. Maar ik schaam mij, ik ben wel egoïstisch, beste Graaf! ik heb er tot hiertoe slechts aan gedacht u mijne eigene grieven te verhalen, en was daarna zoo verdiept in het luisteren naar u, dat ik er bij vergat, hoe vermoeid gij moet zijn en hoe verlangend naar eenige verkwikking.’']

In [7]:
begin_anchor = all_textelements.split(sample_para['begin_anchor'], 11)
end_anchor = all_textelements.split(begin_anchor, 8)

In [8]:
all_textelements.slice(begin_anchor, begin_anchor)

['Geertrui']

In [9]:
all_annotations.append({'label':'entity','begin_anchor': begin_anchor,'end_anchor': begin_anchor,\
                    'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'per', 'entity_text': 'Geertrui'})

In [10]:
entities = [ents for ents in asearch.get_annotations_of_type('entity', all_annotations)]
entities

[{'label': 'entity',
  'begin_anchor': anchor_aa2bba98-6610-4332-955d-444c923d22c3,
  'end_anchor': anchor_aa2bba98-6610-4332-955d-444c923d22c3,
  'id': 'annot_4683bb24-f868-4157-b6b5-7958725b0874',
  'entity_type': 'per',
  'entity_text': 'Geertrui'}]

In [11]:
# print all annotations overlapping with the entity
for a in asearch.get_annotations_overlapping_with(entities[0]['begin_anchor'],entities[0]['end_anchor'],all_annotations):
    print(a)

{'label': 'page', 'begin_anchor': anchor_2f6dabdb-b087-4590-9746-1b5081e34ac3, 'end_anchor': anchor_4d806fe4-5096-42c3-a8bf-46e5239fd060, 'id': 'page-22'}
{'label': 'section', 'begin_anchor': anchor_cb0ed06b-c6d0-4e30-96d2-e5545e9ec7f5, 'end_anchor': anchor_b71373c4-b1f3-4cc3-8d4a-6cb1beb90b3c, 'id': 'annot_59720828-1ac4-4f44-9e43-0210a8dd1538'}
{'label': 'chapter', 'begin_anchor': anchor_cb0ed06b-c6d0-4e30-96d2-e5545e9ec7f5, 'end_anchor': anchor_b71373c4-b1f3-4cc3-8d4a-6cb1beb90b3c, 'id': 'annot_0610d609-66af-487f-b539-ffc40b5cccbe'}
{'label': 'entity', 'begin_anchor': anchor_aa2bba98-6610-4332-955d-444c923d22c3, 'end_anchor': anchor_aa2bba98-6610-4332-955d-444c923d22c3, 'id': 'annot_4683bb24-f868-4157-b6b5-7958725b0874', 'entity_type': 'per', 'entity_text': 'Geertrui'}


Gebruik bovenstaande voor het visualiseren van alle tekst voor een pagina, of alle heads (vanaf het midden van het boek)

In [12]:
for a in asearch.get_annotations_of_type('page', all_annotations):
    if a['id'] == 'page-116':
        print(a)

{'label': 'page', 'begin_anchor': anchor_735ea052-601a-428a-88a7-8ce7dab6f773, 'end_anchor': anchor_5e327823-ac3d-4157-9dff-07feaeb3f079, 'id': 'page-116'}


In [13]:
p116 = [p for p in asearch.get_annotations_of_type('page', all_annotations) if p['id'] == 'page-116'][0]

for t in all_textelements.slice(p116['begin_anchor'],p116['end_anchor']):
    print(f"{t}\n")

dedigen. Het ware hem wellust geweest, slechts eens den zoeten naam der geliefde voor een vertrouwd oor te mogen uitspreken. En toch moest hij laaghartig en zelfzuchtig schijnen, en toch gevoelde hij dat de ander niet meer gelooven kon aan de goedheid van zijn hart, toen hij antwoordde, zooals zijn gegeven woord het eischte:

‘Neen, Arundel, neen! ik kan haar niet helpen; voor haar vermag ik niets; vooralsnog niets;... misschien in het vervolg;... wie weet.... geloof mij, mijn vriend, laat ons geduld hebben.’

‘Dus heeft de beklagenswaardige ook u verloren,’ hervatte de Groot-Kamerheer koel en met stugheid. ‘Ik moet u nog met eene vraag lastig vallen, Mylord; het is de laatste over dit onderwerp. Zult gij mij tegen zijn, zoo ik in haar voordeel spreek?’

‘Op mijn ridderwoord, neen!’ riep Devonshire schielijk; zijne oogen schitterden van vreugde over eene uitkomst, die zoozeer met zijne wenschen instemde en hij stond op en vatte Arundel bij de hand. Zoo brandend jaagde het bloed hem doo

In [15]:
import json

segtextJSONData = json.dumps(all_textelements, indent=4, cls=segmentedtext.SegmentEncoder)
print(segtextJSONData)

{
    "_ordered_segments": [
        "GEBRUIKT EXEMPLAAR",
        "eigen exemplaar dbnl",
        "",
        "ALGEMENE OPMERKINGEN",
        "Dit bestand biedt, behoudens een aantal hierna te noemen ingrepen, een\n                    diplomatische weergave van",
        "De graaf van Devonshire",
        "van A.L.G. Bosboom-Toussaint in een ongedateerde herdruk van ca. 1900. De eerste druk is uit 1838. Deze uitgave bevat ook de novellen",
        "Eene familie-legende",
        "(eerste druk 1864),",
        "De Alkmaarsche wees",
        "(eerste druk 1850) en",
        "Een nacht in een armstoel",
        "(niet los verschenen, eerste druk  in verhalenbundel",
        "De Alkmaarsche wees en eenige andere novellen",
        "in 1854).",
        "",
        "REDACTIONELE INGREPEN",
        "Bij de omzetting van de gebruikte bron naar deze publicatie in de dbnl is een aantal delen van de tekst niet overgenomen. Hieronder volgen de tekstgedeelten die wel in het origineel voorkomen maa

In [16]:
# write all_textlines to a file
with open(datadir+'all_tei_textlines.json', 'w') as filehandle:
    json.dump(all_textelements, filehandle, cls=segmentedtext.SegmentEncoder)

In [17]:
# write all_annotations to a file
with open(datadir+'all_annotations.json', 'w') as filehandle:
    json.dump(all_annotations, filehandle, cls=segmentedtext.AnchorEncoder)

In [18]:
from annotation import aservice
from flask import Flask
import requests
import json

In [20]:
headers = {'Content-Type': 'application/json'}
adata = json.dumps(all_annotations, cls=segmentedtext.AnchorEncoder)
response = requests.put("http://localhost:5000/annotations", data = adata, headers=headers)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /annotations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbad8539c90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [21]:
response = requests.get("http://localhost:5000")

In [22]:
response.json()

{'message': 'un-t-ann-gle Flask annotation service'}